<a href="https://colab.research.google.com/github/tatooland/tensorflow_v2/blob/master/tensorflowV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras.layers as layers
print(tf.__version__)

#tf.enable_eager_execution()

##创建简单网络

In [0]:
inputs = tf.keras.Input(shape=(784,), name='img')
h1 = layers.Dense(32, activation='relu')(inputs)
h2 = layers.Dense(32, activation='relu')(h1)
outputs = layers.Dense(10, activation='softmax')(h2)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

model.summary()

tf.keras.utils.plot_model(model, 'mnist_model.png')
tf.keras.utils.plot_model(model, 'model_info.png', show_shapes=True)

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=400, epochs=20, validation_split=.2)
test_scores = model.evaluate(x_test, y_test, verbose=0)

print('test loss:', test_scores[0])
print('test acc:', test_scores[1])

#使用共享网络创建多个模型

#DCGAN

##define google driver file downloader

In [0]:
!pip install -U -q PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def downloadFromGDriver(fileId):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  fileName = fileId + '.zip'
  downloaded = drive.CreateFile({'id': fileId})
  downloaded.GetContentFile(fileName)
  ds = ZipFile(fileName)
  ds.extractall()
  os.remove(fileName)
  print('Extracted zip file ' + fileName)


In [0]:
downloadFromGDriver("1dCEGqATdkr5uMi22TI1IYFN7NjK9V4oh")

In [0]:
%ls

##读取数据集

In [0]:
import tensorflow as tf
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np


def get_img_path(parentNodePath):
  return listdir(parentNodePath)

def parse_image(fileName):
  print(fileName)
  image = tf.io.read_file(fileName)
  image = tf.image.decode_png(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [128, 128])
  
  return image

def show(image):
  print(image.shape)
  plt.figure()
  plt.imshow(image)

images_path = ["/content/images/%s" % img_path for img_path in get_img_path("/content/images/") if isfile("/content/images/%s" % img_path)]
image_ds = tf.data.Dataset.from_tensor_slices((np.array(images_path)))
image_ds = image_ds.map(parse_image)

print(int(len(images_path)))

train_ds = image_ds.batch(int(len(images_path) * .3)).repeat()
test_ds = image_ds.batch(int(len(images_path) * .3)).repeat()

BATCH_SIZE = int(len(images_path) * .3)
print(BATCH_SIZE)
#show(parse_image('/content/images/wooper.png'))


##generator

In [0]:
def generator():
  #target_shape = (H, W, C) = (128, 128, 3)
  
  input_val = tf.keras.Input(shape=(100,), name='noise')
  input = layers.Dense(4 * 4 * 1024, input_shape=(100,), use_bias=False)(input_val)
  input = layers.LeakyReLU()(input)

  g = layers.Reshape((4, 4, 1024))(input)

  g = layers.Conv2DTranspose(512, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 8, 8, 512)

  g = layers.Conv2DTranspose(256, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 16, 16, 256)

  g = layers.Conv2DTranspose(128, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 32, 32, 128)

  g = layers.Conv2DTranspose(64, 3, strides=(2, 2), use_bias=False, padding='same')(g)
  g = layers.BatchNormalization()(g)
  g = layers.LeakyReLU()(g)
  #assert g.shape() == (None, 64, 64, 64)

  output = layers.Conv2DTranspose(3, 3, strides=(2, 2), use_bias=False, activation='tanh', padding='same')(g)
  #assert g.shape() == (None, 128, 128, 3)

  return tf.keras.Model(inputs=input_val, outputs=output)





###生成图片

In [0]:
import PIL
import matplotlib.pyplot as plt
noise = tf.random.normal([1, 100])
g = generator()
g_img = g(noise, training=False)
print(g_img.shape)
plt.imshow(g_img[0, :, :, 0], cmap='gray' )

##DISCRIMINATOR

In [0]:
def discriminator():
  input_val = tf.keras.Input(shape=(128, 128, 3))
  input = layers.Conv2D(64, 3, strides=(2, 2), padding='same')(input_val)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(64, 64, 64)

  input = layers.Conv2D(128, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(32, 32, 128)

  input = layers.Conv2D(256, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(16, 16, 256)

  input = layers.Conv2D(512, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(8, 8, 512)

  input = layers.Conv2D(1024, 3, strides=(2, 2), padding='same')(input)
  input = layers.LeakyReLU()(input)
  input = layers.Dropout(.3)(input)
  #(4, 4, 1024)

  input = layers.Flatten()(input)
  input = layers.Dense(4*4*1024)(input)
  output = layers.Dense(1)(input)

  return tf.keras.Model(inputs=input_val, outputs=output)

##DISCRIMINATOR LOSS

In [0]:
def combined_loss(d_real_output, d_fake_output):
  bce = tf.keras.losses.BinaryCrossentropy()
  dr_loss = bce(tf.ones_like(d_real_output), d_real_output)
  df_loss = bce(tf.zeros_like(d_fake_output), d_fake_output)
  return dr_loss + df_loss

##GENERATOR LOSS

In [0]:
def generator_loss(g_fake_output):
  bce = tf.keras.losses.BinaryCrossentropy();
  return bce(tf.ones_like(g_fake_output), g_fake_output)

##OPTIMIZER

In [0]:
def optimizer():
  g_optimizer = tf.keras.optimizers.Adam(1e-4)
  d_optimizer = tf.keras.optimizers.Adam(1e-4)
  return g_optimizer, d_optimizer

##TRAIN STEP

In [0]:
#@tf.function
def train_step(images, noise_dim, BATCH_SIZE):
  noise = tf.random.normal([BATCH_SIZE, noise_dim])
  with tf.GradientTape() as g_grad_tape, tf.GradientTape() as d_grad_tape:
    g = generator()
    d = discriminator()
    g_fake_images = g(noise, training=True)
    
    d_fake_output = d(g_fake_images, training=True)
    d_real_output = d(images, training=True)

    d_loss = combined_loss(d_real_output, d_fake_output)
    g_loss = generator_loss(d_fake_output)

  dg = g_grad_tape.gradient(g_loss, g.trainable_variables)
  dd = d_grad_tape.gradient(d_loss, d.trainable_variables)

  g_opt, d_opt = optimizer()

  g_opt.apply_gradients(zip(dg, g.trainable_variables))
  d_opt.apply_gradients(zip(dd, d.trainable_variables))
  return g_fake_images

##EPOCH

In [0]:
import os
import tensorflow as tf

EPOCHS = 50
noise_dim = 100
epoch = 0

#g_opt, d_opt = optimizer()
#g = generator()
#d = discriminator()

#checkpoint_dir = './training_checkpoints'
#checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
#checkpoint = tf.train.Checkpoint(generator_optimizer=g_opt, discriminator_optimizer=d_opt, generator=g, discriminator=d)

for i in range(EPOCHS):
  for img_batch in train_ds:
    imgs = train_step(img_batch, noise_dim, BATCH_SIZE)
    show(imgs[BATCH_SIZE - 1])
  #if epoch % 15 == 0:
    #checkpoint.save(file_prefix = checkpoint_prefix)
    

##define keras image geneerator

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

def readImgDataAsFlow(train_dir, test_dir, validation_dir, target_size=(128, 128)):
  trainGenerator = ImageDataGenerator(data_format="channels_last")
  testGenerator = ImageDataGenerator(data_format="channels_last")
  validationGenerator = ImageDataGenerator(data_format="channels_last")
  trainFlow = trainGenerator.flow_from_directory(train_dir, class_mode='categorical', target_size=target_size)
  testFlow = testGenerator.flow_from_directory(test_dir, class_mode='categorical', target_size=target_size)
  validationFlow = validationGenerator.flow_from_directory(validation_dir, class_mode='categorical', target_size=target_size)
  return trainFlow, testFlow, validationFlow
  

##保存模型和序列化

In [0]:
model.save('model_save.h5')
del model
model = tf.keras.models.load_model('model_save.h5')

In [0]:
encode_input = tf.keras.Input(shape=(28, 28, 1), name='img')
h1 = layers.Conv2D(16, 3, activation='relu')(encode_input)
h1 = layers.Conv2D(32, 3, activation='relu')(h1)
h1 = layers.MaxPool2D(3)(h1)
h1 = layers.Conv2D(32, 3, activation='relu')(h1)
h1 = layers.Conv2D(16, 3, activation='relu')(h1)
encode_output = layers.GlobalMaxPool2D(h1)

encode_model = tf.keras.Model(inputs=encode_input, outputs=encode_output)
encode_model.summary()

h2 = layers.Reshape((4, 4, 1))(encode_output)
h2 = layers.Conv2DTranspose(16, 3, activation='relu')(h2)
h2 = layers.Conv2DTranspose(32, 3, activation='relu')(h2)

h2 = layers.UpSampling2D(3)(h2)
h2 = layers.Conv2DTranspose(16, 3, activation='relu')(h2)
decode_output = layers.Conv2DTranspose(1, 3, activation='relu')(h2)

autoencoder = keras.Model(inputs=encode_input, outputs=decode_output)
autoencoder.summary()